In [1]:
import json

with open("train_item.json") as training:
    training = json.loads(training.read())
    
with open("test_item.json") as testing:
    testing = json.loads(testing.read())

In [2]:
training_dict = {}
testing_dict = {}

for user in training.keys():
    for [item, rating] in training[user]:
        if user in training_dict:
            training_dict[user][item] = rating
        else:
            training_dict[user] = {item: rating}
            
for user in testing.keys():
    for [item, rating] in testing[user]:
        if user in testing_dict:
            testing_dict[user][item] = rating
        else:
            testing_dict[user] = {item: rating}



In [3]:
def sub_row_mean(ratings_dict):
    sub_row_mean_ratings_dict = {}
    for user in ratings_dict:
        list_item = list(ratings_dict[user].values())
        avg = sum(list_item) / len(list_item)
        for item in ratings_dict[user].keys():
            if user in sub_row_mean_ratings_dict:
                sub_row_mean_ratings_dict[user][item] = ratings_dict[user][item] - avg
            else:
                sub_row_mean_ratings_dict[user] = {item: ratings_dict[user][item] - avg}
    
    return sub_row_mean_ratings_dict
        


In [4]:
import numpy as np
from numpy.linalg import norm
import math

def cosine_sim(ratings_dict, user):
    cosine_sim_val = {}

    for other_user in ratings_dict.keys():
        dot_prod = sum(ratings_dict[user][item]*ratings_dict[other_user].get(item, 0) for item in ratings_dict[user])
        user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[user].values()]))
        other_user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[other_user].values()]))
        norm_mul = user_norm * other_user_norm
        if norm_mul == 0:
            cos_sim = 0
        else:
            cos_sim = dot_prod / norm_mul
        if math.isnan(cos_sim):
            cos_sim = 0
        cosine_sim_val[other_user] = cos_sim
    
        if cosine_sim_val[other_user] > 1:
            cosine_sim_val[other_user] = 1
        if cosine_sim_val[other_user] < 0:

            cosine_sim_val[other_user] = 0        
    
    return cosine_sim_val

In [5]:

def predict_item_rating_avg(ratings_dict, cos_similarities_dict, k, item, user):

    cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]

    avg_rating = 0
    
    if len(list_cos_sim) != 0 and list_cos_sim[0][1] != 0:
        for pair in list_cos_sim:
            avg_rating += ratings_dict[pair[0]][item]
        
        avg_rating = avg_rating / len(list_cos_sim)
    else:
        total = 0
        rating = 0
        for rat in training_dict[user]:
            rating += training_dict[user][rat]
            total += 1
        rating = rating / total
        return rating
    
    return avg_rating


    
    

In [6]:


def predict_item_rating_sim(ratings_dict, cos_similarities_dict, k, item, user):

    if user in cos_similarities_dict:
        cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    rating = 0
    similarities = 0
    
    if len(list_cos_sim) != 0 and list_cos_sim[0][1] != 0:
        for pair in list_cos_sim:
            rating += ratings_dict[pair[0]][item] * cos_similarities_dict[pair[0]]
            similarities += cos_similarities_dict[pair[0]]
        rating = rating / similarities
    else:
        total = 0
        rating = 0
        for rat in training_dict[user]:
            rating += training_dict[user][rat]
            total += 1
        rating = rating / total
        
    if rating > 5.1:
        print("Flag")
        print("")
        rating = 4.99
    return rating

  
    

In [7]:
def predict(ratings_dict, user, items):
    predicted_ratings_dict = {}
    sub_mean_dict = sub_row_mean(ratings_dict)
    cos_sim_dict = cosine_sim(sub_mean_dict, user)
    for item in items:
        avg_rating_prediction = predict_item_rating_avg(training_dict, cos_sim_dict.copy(), k, item, user)
        sim_rating_prediction = predict_item_rating_sim(training_dict, cos_sim_dict.copy(), k, item, user)
        
        predicted_ratings_dict[item] = (avg_rating_prediction, sim_rating_prediction)
        
    return predicted_ratings_dict

In [8]:
def first_one_in_dict(dict_first_one):
    count = 0
    for item,rating in dict_first_one.items():
        if rating == 1:
            return count
        else:
            count+=1
    return -1


In [9]:
def ndcg(recommended_items_ratings_avg, recommended_items_ratings_sim, user):
    orig_reviewed_items = (sorted(testing_dict[user].items(), key=lambda x: x[1], reverse=True)).keys()
    orig_reviewed_items_hit_avg = {}
    orig_reviewed_items_hit_sim = {}
    
    for item in orig_reviewed_items:
        orig_reviewed_items_hit_avg[item] = 0
        orig_reviewed_items_hit_sim[item] = 0
    
    recommended_items_avg = []#recommended_items_ratings_avg.keys()
    recommended_items_sim = []#recommended_items_ratings_sim.keys()

    for item in recommended_items_ratings_avg:
        recommended_items_avg.append(item[0])

    for item in recommended_items_ratings_sim:
        recommended_items_sim.append(item[0])

    same_avg_items = 0
    same_sim_items = 0

    for item in recommended_items_avg:
        if item in orig_reviewed_items:
            same_avg_items += 1
            orig_reviewed_items_hit_avg[item] = 1
            
            

    for item in recommended_items_sim:
        if item in orig_reviewed_items:
            same_sim_items += 1
            orig_reviewed_items_hit_sim[item] = 1
            
    dcg_avg = 0
    idcg_avg = 0
    ndcg_avg = 0
    if same_avg_items == 0:
        ndcg_avg = 0
    else:
        first_one = first_one_in_dict(orig_reviewed_items_hit_avg)
        for i in range(same_avg_items):
            dcg_avg += (1/math.log2(1 + first_one + i + 1))
            idcg_avg += (1/math.log2(1 + i + 1))
            
        ndcg_avg = dcg_avg/idcg_avg
        
        
    dcg_sim = 0
    idcg_sim = 0
    ndcg_sim = 0
    if same_sim_items == 0:
        ndcg_sim = 0
    else:
        first_one = first_one_in_dict(orig_reviewed_items_hit_sim)
        for i in range(same_sim_items):
            dcg_sim += (1/math.log2(1 + first_one + i + 1))
            idcg_sim += (1/math.log2(1 + i + 1))
            
        ndcg_sim = dcg_sim/idcg_sim
    print('NDCG Avg: ')
    print(ndcg_avg)
    print('NDCG Sim: ')
    print(ndcg_sim)
   
    


In [10]:
def scores(recommended_items_ratings_avg, recommended_items_ratings_sim, user):



    print((user, recommended_items_ratings_avg))
    print((user, recommended_items_ratings_sim))

    orig_reviewed_items = testing_dict[user].keys()
    recommended_items_avg = []#recommended_items_ratings_avg.keys()
    recommended_items_sim = []#recommended_items_ratings_sim.keys()
    recommended_items_pair_avg = {}#recommended_items_ratings_avg.keys()
    recommended_items_pair_sim = {}#recommended_items_ratings_sim.keys()
    

    for item in recommended_items_ratings_avg:
        recommended_items_avg.append(item[0])
        recommended_items_pair_avg[item[0]] = 0 
        

    for item in recommended_items_ratings_sim:
        recommended_items_sim.append(item[0])
        recommended_items_pair_sim[item[0]] = 0 

    same_avg_items = 0
    same_sim_items = 0

    for item in recommended_items_avg:
        if item in orig_reviewed_items:
            same_avg_items += 1
            recommended_items_pair_avg[item] = 1

    for item in recommended_items_sim:
        if item in orig_reviewed_items:
            same_sim_items += 1
            recommended_items_pair_sim[item] = 1

    precision_avg = (same_avg_items/k)
    precision_sim = (same_sim_items/k)

    print("Precision Avg: ")
    print(precision_avg)
    print('Precision Sim: ')
    print(precision_sim)


    recall_avg = (same_avg_items/len(orig_reviewed_items))
    recall_sim = (same_sim_items/len(orig_reviewed_items))

    print('Recall Avg: ')
    print(recall_avg)
    print('Recall Sim: ')
    print(recall_sim)
    if precision_avg+recall_avg == 0:
        f_score_avg = 0
    else:
        f_score_avg = (2*precision_avg*recall_avg)/(precision_avg+recall_avg)

    if precision_sim+recall_sim == 0:
        f_score_sim = 0
    else:
        f_score_sim = (2*precision_sim*recall_sim)/(precision_sim+recall_sim)

    print('F_Score Avg: ')
    print(f_score_avg)
    print('F_Score Sim: ')
    print(f_score_sim)
    print(testing_dict[user])
    

    
    dcg_avg = 0
    idcg_avg = 0
    ndcg_avg = 0
    if same_avg_items != 0:
        first_one = first_one_in_dict(recommended_items_pair_avg)
        for i in range(same_avg_items):
            dcg_avg += (1/math.log2(1 + first_one + i + 1))
            idcg_avg += (1/math.log2(1 + i + 1))
            
        ndcg_avg = dcg_avg/idcg_avg
        
        
    dcg_sim = 0
    idcg_sim = 0
    ndcg_sim = 0
    if same_sim_items != 0:
        first_one = first_one_in_dict(recommended_items_pair_sim)
        for i in range(same_sim_items):
            dcg_sim += (1/math.log2(1 + first_one + i + 1))
            idcg_sim += (1/math.log2(1 + i + 1))
            
        ndcg_sim = dcg_sim/idcg_sim
    print('NDCG Avg: ')
    print(ndcg_avg)
    print('NDCG Sim: ')
    print(ndcg_sim)
    
    return(precision_avg, precision_sim, recall_avg, recall_sim, f_score_avg, f_score_sim, ndcg_avg, ndcg_sim)
   
    

    
    
    
    
    
    
    
    

In [11]:
all_items = set()
for user,list_ratings in training_dict.items():
    all_items.update(list(training_dict[user].keys()))

for user,list_ratings in testing_dict.items():
    all_items.update(list(testing_dict[user].keys()))


In [12]:
top_n_testing_dict = dict(sorted(testing_dict.items(), key=lambda x: len(x[1]), reverse=True)[:100])

In [14]:
import time

k = 10


prec_avg, prec_sim, rec_avg, rec_sim, f_avg, f_sim, ndcgav, ndcgsim = 0, 0, 0, 0, 0, 0, 0, 0

start = time.time()
i = 0
j = 0

for user, list_ratings in top_n_testing_dict.items():
    i += 1
    predicted_vals = predict(training_dict, user, list(all_items))

    orig_vals = training_dict[user]

    for item,rating in orig_vals.items():

        predicted_vals.pop(item)
        
    print('User Count: ' + str(i))
    print(time.time() - start)


    recommended_items_ratings_avg = sorted(predicted_vals.items(), key=lambda x: x[1][0], reverse=True)[:k]
    recommended_items_ratings_sim = sorted(predicted_vals.items(), key=lambda x: x[1][1], reverse=True)[:k]

    temp_prec_avg, temp_prec_sim, temp_rec_avg, temp_rec_sim, temp_f_avg, temp_f_sim, temp_ndcgav, temp_ndcgsim = scores(recommended_items_ratings_avg, recommended_items_ratings_sim, user)
    
    prec_avg += temp_prec_avg
    prec_sim += temp_prec_sim
    rec_avg += temp_rec_avg
    rec_sim += temp_rec_sim
    f_avg += temp_f_avg
    f_sim += temp_f_sim
    ndcgav += temp_ndcgav
    ndcgsim += temp_ndcgsim




end = time.time()

print(end - start) 

print('Total Precision Average: ')
print(prec_avg/100)
print('Total Precision Similar: ')
print(prec_sim/100)
print('Total Recall Average: ')
print(rec_avg/100)
print('Total Recall Similar: ')
print(rec_sim/100)
print('Total F Score Average: ')
print(f_avg/100)
print('Total F Score Similar: ')
print(f_sim/100)
print('NDCG Average: ')
print(ndcgav/100)
print('NDCG Similar: ')
print(ndcgsim/100)

User Count: 1
454.4498302936554
('B00CZF8B68', [('A2DA2SP0WJ026A', (5.0, 5.0)), ('A2NSFXAVIDKJLY', (5.0, 5.0)), ('A3P6VQPKKIPFIZ', (5.0, 5.0)), ('A3NQKILELEP02H', (5.0, 5.0)), ('A1AGIP0FLI2VZW', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A1DQPVC165HWND', (5.0, 5.0)), ('A14G6J1V2NQ4XJ', (5.0, 5.0)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('AKU7YAU8IS8I4', (5.0, 5.0))])
('B00CZF8B68', [('A2NRH0W1INVZV4', (5.0, 5.000000000000003)), ('A2U9TMU40ZJWBL', (5.0, 5.000000000000003)), ('A3AJIQEVA6T0GQ', (5.0, 5.000000000000003)), ('A351338QZ9X5OW', (5.0, 5.000000000000002)), ('A2HHKXLV3Z6O7Z', (5.0, 5.000000000000002)), ('A2T952ZQXI06KJ', (5.0, 5.000000000000002)), ('A3OCU2UDKQKW97', (4.625, 5.000000000000002)), ('A2X8ORD8P2R39S', (5.0, 5.000000000000002)), ('A12PWC358D76IZ', (5.0, 5.000000000000002)), ('AJI7Q53V06VPR', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1US7BFCSQKQ70': 5.0, 'A2CC0A0H0DDZQR': 5.0, '

User Count: 3
1315.4177329540253
('B00BWGHIHY', [('A2DA2SP0WJ026A', (5.0, 5.0)), ('A2NSFXAVIDKJLY', (5.0, 5.0)), ('A2OGJ0HAS9QWKM', (5.0, 5.0)), ('A3NQKILELEP02H', (5.0, 5.0)), ('A1AGIP0FLI2VZW', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('AKU7YAU8IS8I4', (5.0, 5.0)), ('A1FONII7J7OEX4', (5.0, 5.0)), ('A2SCF9W9JXI6EQ', (5.0, 5.0))])
('B00BWGHIHY', [('A36O4M7Z7L5IFA', (4.7, 5.000000000000002)), ('A2V5XFNXPUQVLS', (5.0, 5.000000000000002)), ('A3JZGPHDBD5CMM', (4.666666666666667, 5.000000000000002)), ('A225KOTGHS3BRA', (5.0, 5.000000000000002)), ('A3O9HF7IXLMGDN', (5.0, 5.000000000000002)), ('A2HDYNSUN6JJUP', (5.0, 5.000000000000002)), ('A1GHUN5HXMHZ89', (4.9, 5.000000000000002)), ('A31YTLNTI9C6U0', (5.0, 5.000000000000002)), ('A281DXZBJSUHPQ', (5.0, 5.000000000000002)), ('A3E98R6OJVOWLX', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A23XDLRMGN5BCS': 5.0, 'A1YL3G7W5

User Count: 6
2600.4953787326813
('B00EH49FRE', [('A2DA2SP0WJ026A', (5.0, 5.0)), ('AY039JD9177ET', (5.0, 5.0)), ('A2NSFXAVIDKJLY', (5.0, 5.0)), ('A3P6VQPKKIPFIZ', (5.0, 5.0)), ('A1AGIP0FLI2VZW', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.000000000000001)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('AKU7YAU8IS8I4', (5.0, 5.0)), ('A2SCF9W9JXI6EQ', (5.0, 5.0)), ('A2XVIKGJ64GNGG', (5.0, 5.0))])
('B00EH49FRE', [('AGTM2VNAL7VS3', (5.0, 5.000000000000003)), ('A2R7PJGG0BWY0J', (5.0, 5.000000000000003)), ('A35P1COO95IH1R', (5.0, 5.000000000000003)), ('A351338QZ9X5OW', (5.0, 5.000000000000002)), ('A3DBDULLZISWMC', (4.8, 5.000000000000002)), ('A32T2H8150OJLU', (5.0, 5.000000000000002)), ('A3O9HF7IXLMGDN', (5.0, 5.000000000000002)), ('AF6LL36EXQ210', (4.8, 5.000000000000002)), ('A3G6G5BJ67W1KP', (5.0, 5.000000000000002)), ('A3DMRA0VAYNUA1', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1Q6MHDUPNV6P7': 5.0, 'A229NX5YKTDZ

User Count: 9
3888.2105779647827
('B0092MKTL2', [('AY039JD9177ET', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A1H88B50FBNZG6', (5.0, 5.0)), ('AWQCW0V7M6KQT', (5.0, 5.0)), ('A2SCF9W9JXI6EQ', (5.0, 5.0)), ('A1HD9GFICWP02B', (5.0, 5.0)), ('A2XVIKGJ64GNGG', (5.0, 5.0)), ('A392QIW5KUJEX2', (5.0, 5.0)), ('A3MOCZ1VXOA8VJ', (5.0, 5.0)), ('A1OH33MR6K4G5J', (5.0, 5.0))])
('B0092MKTL2', [('ABCXJIK7QMLA7', (5.0, 5.000000000000003)), ('A3T13H29NQEJOI', (5.0, 5.000000000000002)), ('A6XMXS05BQ6MK', (5.0, 5.000000000000002)), ('A3GYWZE27QBMY7', (5.0, 5.000000000000002)), ('A35GU60F0WQJ7B', (4.833333333333333, 5.000000000000002)), ('A1F37SJMP29EMF', (4.888888888888889, 5.000000000000002)), ('AS0CCKP5ZB0M6', (5.0, 5.000000000000002)), ('A1GR5SRP9SLDPA', (5.0, 5.000000000000002)), ('A19GXRXQCP82ES', (5.0, 5.000000000000001)), ('A1TEP5YH4CDDSU', (4.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1X2S2BF67JJL7': 5.0, 

User Count: 13
6199.851373195648
('B0092MKTWQ', [('A3P6VQPKKIPFIZ', (5.0, 5.0)), ('A1AGIP0FLI2VZW', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A38Z1I1CCH6HDO', (5.0, 5.0)), ('AWQCW0V7M6KQT', (5.0, 5.0)), ('A1FONII7J7OEX4', (5.0, 4.999999999999999)), ('A2SCF9W9JXI6EQ', (5.0, 5.0)), ('A2XVIKGJ64GNGG', (5.0, 4.999999999999999)), ('A3EV1YFMIHBY3P', (5.0, 5.0)), ('A1I72OXASRG2DB', (5.0, 5.0))])
('B0092MKTWQ', [('A20173T9HDB759', (5.0, 5.000000000000003)), ('A6XHF110BC856', (5.0, 5.000000000000003)), ('AXND5887INISD', (5.0, 5.000000000000002)), ('A1LUGVFDKPE7N7', (5.0, 5.000000000000002)), ('A2IE5VF6N4R2FU', (5.0, 5.000000000000002)), ('ASABX35QYUEZL', (5.0, 5.000000000000002)), ('A142HAT43L5W4W', (5.0, 5.000000000000002)), ('A2J94RBT2849R', (5.0, 5.000000000000002)), ('A24SAA1Y7UULLT', (5.0, 5.000000000000002)), ('A5PZNLSCMM4KQ', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'AAFGG6R921K53': 5.0, 'A

...

User Count: 80
34795.15329384804
('B01H5ZYNM8', [('A3P6VQPKKIPFIZ', (5.0, 5.0)), ('A1AGIP0FLI2VZW', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('AKU7YAU8IS8I4', (5.0, 4.999999999999999)), ('A2SCF9W9JXI6EQ', (5.0, 5.0)), ('A2XVIKGJ64GNGG', (5.0, 4.999999999999999)), ('A2CK4EU596B5TJ', (5.0, 5.0)), ('A27CG68MDW38IW', (5.0, 5.0)), ('A33H4A43NUZOLY', (5.0, 5.0))])
('B01H5ZYNM8', [('A3VP9JGZNO78JP', (5.0, 5.000000000000002)), ('A1MSO3EK95SVCF', (5.0, 5.000000000000002)), ('A2R819E0C80FXW', (5.0, 5.000000000000002)), ('A13G2LMKA9GVMM', (5.0, 5.000000000000002)), ('A2QVXVZ8DNAMRU', (5.0, 5.000000000000002)), ('A3PJ5FGZWHRYTI', (5.0, 5.000000000000002)), ('A2GEURP6P1FIHR', (4.8, 5.000000000000002)), ('A27YS4JDS7J3IC', (4.9, 5.000000000000002)), ('A3AWB8K41TZQGX', (5.0, 5.000000000000002)), ('A2DPMXWJR44KLH', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A2D8V4W04371GQ': 5

User Count: 85
36918.288120508194
('B0011Z1D8Q', [('A1AGIP0FLI2VZW', (5.0, 5.0)), ('A1DQPVC165HWND', (5.0, 5.0)), ('AKU7YAU8IS8I4', (5.0, 5.0)), ('A3EV1YFMIHBY3P', (5.0, 5.0)), ('A3FY3S8KMC5X82', (5.0, 5.0)), ('A30AK12YSTRA0Q', (5.0, 5.0)), ('A1J7OIXQFXVQ7P', (5.0, 5.0)), ('AQG39LCFBIAGA', (5.0, 5.000000000000001)), ('A3VU112V9W8Q4A', (5.0, 5.0)), ('A1IKP1UHJVW5JF', (5.0, 5.0))])
('B0011Z1D8Q', [('A1MCJ0LH5N330H', (5.0, 5.000000000000002)), ('A3EQLH3KZQ40ZC', (4.9, 5.000000000000002)), ('A1OKPC1K12YZEP', (5.0, 5.000000000000002)), ('ARYVQL4N737A1', (5.0, 5.000000000000002)), ('A15T3Y3BL2AJ9V', (5.0, 5.000000000000002)), ('A3DLFHSYRIIXYO', (5.0, 5.000000000000002)), ('A1Y0OW8T5SM0TJ', (5.0, 5.000000000000002)), ('AQG39LCFBIAGA', (5.0, 5.000000000000001)), ('AHPRLL52R0TZZ', (5.0, 5.000000000000001)), ('A19D4GEQH3Q5OU', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'AP2OA75LGQI4Z': 5.0, 'A3LO89HQ6KQLE

User Count: 90
39037.730828523636
('B005VPSEVG', [('A2DA2SP0WJ026A', (5.0, 5.0)), ('A2NSFXAVIDKJLY', (5.0, 5.0)), ('ACI57ZVTAWSNY', (5.0, 4.999999999999999)), ('A1AGIP0FLI2VZW', (5.0, 5.000000000000001)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A38Z1I1CCH6HDO', (5.0, 5.0)), ('A14G6J1V2NQ4XJ', (5.0, 5.0)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('A1H88B50FBNZG6', (5.0, 5.0)), ('AKU7YAU8IS8I4', (5.0, 5.0))])
('B005VPSEVG', [('A2HO0IEDD64204', (5.0, 5.000000000000003)), ('AF9NWZ4SJ5ZUU', (5.0, 5.000000000000003)), ('A2LAR13V99X62U', (5.0, 5.000000000000003)), ('A2YW2W4WXU53QL', (5.0, 5.000000000000003)), ('A1M3F4G45OXRDI', (5.0, 5.000000000000003)), ('A2TXKUC8SN53H7', (5.0, 5.000000000000003)), ('AYHIJ942LTGZM', (5.0, 5.000000000000002)), ('A3M86N46CZR3RG', (5.0, 5.000000000000002)), ('A3G6G5BJ67W1KP', (5.0, 5.000000000000002)), ('AB4GT7VJMB4A3', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A38HMZ7KPUA8OM': 5.0,

User Count: 95
41164.97375845909
('B0013D6QP6', [('A2DA2SP0WJ026A', (5.0, 5.0)), ('A2NSFXAVIDKJLY', (5.0, 5.0)), ('A3P6VQPKKIPFIZ', (5.0, 5.0)), ('A1VUA80DLZGWIP', (5.0, 5.0)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('A1H88B50FBNZG6', (5.0, 4.999999999999999)), ('AKU7YAU8IS8I4', (5.0, 5.0)), ('A1FONII7J7OEX4', (5.0, 5.0)), ('A2SCF9W9JXI6EQ', (5.0, 5.0)), ('A2XVIKGJ64GNGG', (5.0, 5.0))])
('B0013D6QP6', [('ANLYV7E27PQ91', (5.0, 5.000000000000003)), ('A2POF7JE20652', (5.0, 5.000000000000003)), ('A2RNYF2PTOB82J', (5.0, 5.000000000000002)), ('A2EDMJSIH8ZUIT', (5.0, 5.000000000000002)), ('AGR0LD8NNZATF', (5.0, 5.000000000000002)), ('A2MWR05ECJBLLH', (5.0, 5.000000000000002)), ('A1GL1F09K9B0UR', (5.0, 5.000000000000002)), ('A1NDKH5MGL8MR1', (5.0, 5.000000000000002)), ('AXFMUQM9B2HTR', (5.0, 5.000000000000002)), ('A15VH7AWS9ZIJK', (5.0, 5.000000000000002))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A1PB97NTX3RO7': 4.0, 'A3K7RJ82F10U8H

User Count: 100
43288.73276948929
('B00QVZ1VXI', [('A1VUA80DLZGWIP', (5.0, 5.0)), ('A1OJPZZQAF9YGI', (5.0, 5.0)), ('A3I29W7R4ARY1E', (5.0, 5.0)), ('A2SCF9W9JXI6EQ', (5.0, 5.0)), ('A1HD9GFICWP02B', (5.0, 5.0)), ('A382OWESG9UBOU', (5.0, 5.0)), ('A3EV1YFMIHBY3P', (5.0, 4.999999999999999)), ('A3ML13JQ3OWEKX', (5.0, 5.0)), ('A29VJT0J44DZX3', (5.0, 5.0)), ('A3MOCZ1VXOA8VJ', (5.0, 5.0))])
('B00QVZ1VXI', [('A30AK12YSTRA0Q', (5.0, 5.000000000000002)), ('A3J74IC5X508RW', (5.0, 5.000000000000001)), ('A1ID8VQWS1EBZX', (5.0, 5.000000000000001)), ('A3QXAIS1DVE35O', (5.0, 5.000000000000001)), ('A19B0AO4E4QE12', (5.0, 5.000000000000001)), ('A1VLJIGQNNCTL5', (5.0, 5.000000000000001)), ('A2HHKXLV3Z6O7Z', (5.0, 5.000000000000001)), ('A2RNYF2PTOB82J', (5.0, 5.000000000000001)), ('A2BMXG9XRU48N7', (5.0, 5.000000000000001)), ('A2Q3DE1LATC2GV', (5.0, 5.000000000000001))])
Precision Avg: 
0.0
Precision Sim: 
0.0
Recall Avg: 
0.0
Recall Sim: 
0.0
F_Score Avg: 
0
F_Score Sim: 
0
{'A3GINGW1GC04OD': 5.0, 'AGR0Y8W